In [119]:
import pandas as pd
import numpy as np
import math

In [120]:
dataset_ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")

In [122]:
stock_symbols_ftse_mib = ["A2A.MI", "AMP.MI", "ATL.MI", "AZM.MI", "BAMI.MI", "BGN.MI", "BMED.MI", "BPE.MI", "BZU.MI", "CNHI.MI", "CPR.MI", "DIA.MI", "ENEL.MI", "ENI.MI", "EXO.MI", "FBK.MI", "G.MI", "HER.MI",
                          "IG.MI", "INW.MI", "IP.MI", "ISP.MI", "LDO.MI", "MB.MI", "MONC.MI", "NEXI.MI", "PIRC.MI", "PRY.MI", "PST.MI", "RACE.MI", "REC.MI", "SPM.MI", "SRG.MI", "STLA.MI", "STM.MI", "TEN.MI", "TIT.MI", "TRN.MI", "UCG.MI", "UNI.MI", "FTSEMIB.MI"]


In [123]:
dataframes = {}
for sym in stock_symbols_ftse_mib:
  frame = pd.read_csv("./Datasets/"+sym+".csv")
  dataframes[sym] = frame

In [124]:
def get_stocks_by_week(df, week):
    """
    Returns a dataframe of stocks for a given week
    """
    return df[df['Week Number'] == week]

In [125]:
def get_portfolio_weekly_return(portfolio, weekNumber):
  totalReturn = 0
  
  for index, row in portfolio.iterrows():
    
    stock_dataframe = pd.read_csv("./Datasets/"+row["Stock Symbol"]+".csv")
    stock_log_return_initial = stock_dataframe.iloc[180+int(
        weekNumber)]["log_ret"] if 180+int(weekNumber) < len(stock_dataframe) else 0
    stock_log_ret_next_day = stock_dataframe.iloc[180+int(weekNumber)+8]["log_ret"] if 180+int(weekNumber)+8 < len(stock_dataframe) else 0
    
    totalReturn += (stock_log_ret_next_day - stock_log_return_initial) / len(portfolio)
  return totalReturn


In [126]:
def get_ftse_mib_weekly_return(week_number):
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  prev_week_ret = ftse_mib.iloc[180+week_number-7]["log_ret"]
  next_week_ret = ftse_mib.iloc[180+week_number]["log_ret"]
  return next_week_ret-prev_week_ret


In [127]:
def get_ftse_mib_risk(week_number): 
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  variance = ftse_mib.iloc[180+week_number-7:180+week_number]["log_ret"].var()
  return math.sqrt(variance)

In [128]:
test = pd.read_csv("./results.csv")

In [129]:
test.head()

,Unnamed: 0,Stock Symbol,Week Number,Alpha,Beta,R-Squared,Risk,Res Std. Error,Stock Weekly Return
0,0,A2A.MI,0,-0.000713,0.779469,0.293579,0.000174,0.007567,-0.018795
1,1,A2A.MI,7,-0.000081,-0.035455,0.000753,0.000279,0.000026,0.003708
2,2,A2A.MI,14,-0.000423,-0.039381,0.000963,0.000132,0.000034,0.019859
3,3,A2A.MI,21,-0.000870,-0.048152,0.001466,0.000078,0.000052,-0.027421
4,4,A2A.MI,28,-0.001095,-0.067737,0.003573,0.000200,0.000146,0.020515


In [130]:
def get_sorted_df(dataframe, sort_method):
  dataframe = dataframe.sort_values(by=sort_method, ascending=False)
  return dataframe


In [131]:
def get_portfolio_risk(portfolio, weekNumber):
  risk = 0
  columns = portfolio["Stock Symbol"].unique()
  filtered_portfolio = pd.DataFrame(columns=columns)
  
  for index, row in portfolio.iterrows():
    filtered_portfolio[row["Stock Symbol"]] = dataframes[row["Stock Symbol"]].iloc[180+weekNumber:180+weekNumber+7]["log_ret"]
  
  portfolio_variance = 0
  cov_matrix = filtered_portfolio.cov()
  for index, row in portfolio.iterrows():
    for index2, row2 in portfolio.iterrows():
      covariance = cov_matrix.loc[row["Stock Symbol"], row2["Stock Symbol"]]
      portfolio_variance+= 2 * (1/len(portfolio)) * covariance
  return math.sqrt(portfolio_variance)


Test: take top 15 and bottom 15 based on beta

In [180]:
final_dataframe = pd.DataFrame(columns=[
                               "Week", "portfolio Weekly Returns Beta", "portfolio Weekly Returns R-Squared", "portfolio Weekly Returns Risk", "portfolio Weekly Returns Res Std Error", "portfolio Weekly Returns Returns", "Ftse Mib weekly returns","Beta Risk","R-Squared Risk","Variance risk", "Residual-Variance Risk", "Return Risk","FTSE_MIB Risk"])
results = pd.read_csv("./results.csv")
for weekNumber in range(0,len(dataset_ftse_mib)-1,7):
  res = get_stocks_by_week(test, weekNumber)
  res = res[res["Stock Symbol"] != "FTSEMIB.MI"]

  sorted_df_beta = get_sorted_df(res, "Beta")
  sorted_df_rsquared = get_sorted_df(res, "R-Squared")
  sorted_df_risk = get_sorted_df(res, "Risk")
  sorted_df_res_std_error = get_sorted_df(res, "Res Std. Error")
  sorted_df_ret = get_sorted_df(res, "Stock Weekly Return")

  frames_beta = [sorted_df_beta.head(4),  sorted_df_beta.tail(4)]
  frames_rsquared = [sorted_df_rsquared.head(4),  sorted_df_rsquared.tail(4)]
  frames_risk = [sorted_df_risk.head(4),  sorted_df_risk.tail(4)]
  frames_res_std_error = [sorted_df_res_std_error.head(
      4),  sorted_df_res_std_error.tail(4)]
  frames_ret = [sorted_df_ret.head(8)]

  top_bottom_10_beta = pd.concat(frames_beta)
  top_bottom_10_rsquared = pd.concat(frames_rsquared)
  top_bottom_10_risk = pd.concat(frames_risk)
  top_bottom_10_res_std_error = pd.concat(frames_res_std_error)
  top_bottom_10_ret = pd.concat(frames_ret)

  risk_beta = get_portfolio_risk(top_bottom_10_beta,weekNumber)
  risk_rsquared = get_portfolio_risk(top_bottom_10_rsquared, weekNumber)
  risk_variance = get_portfolio_risk(top_bottom_10_risk, weekNumber)
  risk_errors = get_portfolio_risk(top_bottom_10_res_std_error, weekNumber)
  risk_returns = get_portfolio_risk(top_bottom_10_ret, weekNumber)
  risk_ftse_mib = get_ftse_mib_risk(weekNumber)
  if len(top_bottom_10_beta) > 0:
    portfolio_return_beta = get_portfolio_weekly_return(
        top_bottom_10_beta, weekNumber)
    portfolio_return_rsquared = get_portfolio_weekly_return(
        top_bottom_10_rsquared, weekNumber)
    portfolio_return_risk = get_portfolio_weekly_return(
        top_bottom_10_risk, weekNumber)
    portfolio_return_res_std_error = get_portfolio_weekly_return(
        top_bottom_10_rsquared, weekNumber)
    portfolio_return_ret = get_portfolio_weekly_return(
        top_bottom_10_ret, weekNumber)
    ftse_mib_return = get_ftse_mib_weekly_return(weekNumber)
    final_dataframe.loc[len(final_dataframe)] = [
        weekNumber, portfolio_return_beta, portfolio_return_rsquared,portfolio_return_risk,portfolio_return_res_std_error,portfolio_return_ret, ftse_mib_return, risk_beta, risk_rsquared, risk_variance, risk_errors, risk_returns,risk_ftse_mib]


In [181]:
res = get_stocks_by_week(test, 707)
res = res[res["Stock Symbol"] != "FTSEMIB.MI"]
sorted_df_ret_test = get_sorted_df(res, "Stock Weekly Return")
get_portfolio_weekly_return(sorted_df_ret_test, 707)


0.007148822725954089

In [182]:
sorted_df_ret_test.head(8)

,Unnamed: 0,Stock Symbol,Week Number,Alpha,Beta,R-Squared,Risk,Res Std. Error,Stock Weekly Return
4857,4857,SPM.MI,707,-0.006356,0.016800,0.000134,0.000461,0.000033,0.030390
3845,3845,MONC.MI,707,-0.001282,-0.141328,0.014754,0.000164,0.002328,0.028906
569,569,AZM.MI,707,-0.002468,-0.069478,0.002591,0.000266,0.000563,0.023913
1193,1193,BPE.MI,707,-0.007014,-0.093186,0.001870,0.000367,0.001012,0.020804
5013,5013,SRG.MI,707,-0.000652,-0.141857,0.015064,0.000057,0.002345,0.016948
2909,2909,IG.MI,707,-0.000337,-0.069373,0.005434,0.000116,0.000561,0.016840
6105,6105,UNI.MI,707,-0.001535,-0.008833,0.000044,0.000305,0.000009,0.016763
2129,2129,ENI.MI,707,-0.004097,-0.013825,0.000094,0.000455,0.000022,0.015860


In [183]:
final_dataframe.drop(final_dataframe.tail(1).index, inplace=True)

In [184]:
final_dataframe.head()

,Week,portfolio Weekly Returns Beta,portfolio Weekly Returns R-Squared,portfolio Weekly Returns Risk,portfolio Weekly Returns Res Std Error,portfolio Weekly Returns Returns,Ftse Mib weekly returns,Beta Risk,R-Squared Risk,Variance risk,Residual-Variance Risk,Return Risk,FTSE_MIB Risk
0,0.0,-0.002092,0.015911,-0.014866,0.015911,-0.044287,-0.008566,0.031108,0.018322,0.028059,0.031108,0.018513,0.011579
1,7.0,0.011727,0.013740,0.015983,0.013740,-0.006460,-0.023240,0.050711,0.058148,0.057152,0.043605,0.018694,0.006071
2,14.0,-0.005981,0.006981,-0.006362,0.006981,-0.021130,0.020911,0.014667,0.034658,0.032512,0.017454,0.025883,0.013868
3,21.0,0.038698,0.037576,0.042674,0.037576,0.004117,-0.015410,0.042844,0.043342,0.055059,0.043157,0.023516,0.008636
4,28.0,-0.005923,0.007779,0.003031,0.007779,-0.044051,0.015685,0.036385,0.032619,0.040168,0.033056,0.044141,0.010359


In [185]:
import plotly.express as px


In [196]:
initial_investment = 100
portfolio_beta_total_return = initial_investment
portfolio_rsquared_total_return = initial_investment
portfolio_risk_total_return = initial_investment
portfolio_res_std_err_total_return = initial_investment
portfolio_returns_total_return = initial_investment
index_total_return = initial_investment
first = True
final_returns = pd.DataFrame(
    columns=["Week", "Beta Portfolio", "R-Squared Portfolio","Variance Portfolio","Residual variance Portfolio","Momentum Portfolio", "FTSE_MIB", "Beta Risk","R-Squared Risk", "Variance Risk","Error Risk","Returns risk"])
for index, row in final_dataframe.iterrows():
  
  portfolio_beta_total_return *= math.exp(row["portfolio Weekly Returns Beta"])
  portfolio_rsquared_total_return *= math.exp(
      row["portfolio Weekly Returns R-Squared"])
  portfolio_risk_total_return *= math.exp(
      row["portfolio Weekly Returns Risk"])
  portfolio_res_std_err_total_return *= math.exp(
      row["portfolio Weekly Returns Res Std Error"])
  portfolio_returns_total_return *= math.exp(
      row["portfolio Weekly Returns Returns"])
  index_total_return *= math.exp(row["Ftse Mib weekly returns"])
 
  final_returns.loc[len(final_returns)] = [row["Week"],
                                           portfolio_beta_total_return, 
                                           portfolio_rsquared_total_return, 
                                           portfolio_risk_total_return, 
                                           portfolio_res_std_err_total_return, 
                                           portfolio_returns_total_return, 
                                           index_total_return,
                                            row["Beta Risk"].mean(), 
                                            row["R-Squared Risk"].mean(), 
                                            row["Variance risk"].mean(), 
                                            row["Residual-Variance Risk"].mean(), 
                                            row["Return Risk"].mean()]
  

In [197]:
final_returns.head(2)

,Week,Beta Portfolio,R-Squared Portfolio,Variance Portfolio,Residual variance Portfolio,Momentum Portfolio,FTSE_MIB,Beta Risk,R-Squared Risk,Variance Risk,Error Risk,Returns risk
0,0.0,99.791037,101.603809,98.524351,101.603809,95.667966,99.147019,0.031108,0.018322,0.028059,0.031108,0.018513
1,7.0,100.968216,103.009470,100.111737,103.009470,95.051925,96.869363,0.050711,0.058148,0.057152,0.043605,0.018694


In [198]:
final_returns["FTSE_MIB"].var()


2.5319899179870893

In [199]:
len(final_returns)

155

## Calculation of portfolio risks and expected returns

Portfolios risks

In [200]:
print("R-Squared portfolio risk: " +
      str(final_returns["R-Squared Risk"].mean()*math.sqrt(52)))
print("Beta portfolio risk: " +
      str(final_returns["Beta Risk"].mean()*math.sqrt(52)))
print("Variance portfolio risk: " +
      str(final_returns["Variance Risk"].mean()*math.sqrt(52)))
print("Res std error portfolio risk: " +
      str(final_returns["Error Risk"].mean()*math.sqrt(52)))
print("Returns portfolio risk: " +
      str(final_returns["Returns risk"].mean()*math.sqrt(52)))
print("FTSE-MIB portfolio risk: " +
      str(final_dataframe["FTSE_MIB Risk"].mean()*math.sqrt(52)))



R-Squared portfolio risk: 0.3251005458482946
Beta portfolio risk: 0.36232613987740847
Variance portfolio risk: 0.35258818341095577
Res std error portfolio risk: 0.350848036196302
Returns portfolio risk: 0.27378747255262714
FTSE-MIB portfolio risk: 0.08630134543054792


### Portfolios expected returns

In [201]:
print("R-Squared portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns R-Squared"].mean()*52))
print("Beta portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Beta"].mean()*52))
print("Variance portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Risk"].mean()*52))
print("Res std error portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Res Std Error"].mean()*52))
print("Returns portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Returns"].mean()*52))
print("FTSE-MIB portfolio expected return: " +
      str(final_dataframe["Ftse Mib weekly returns"].mean()*52))


R-Squared portfolio expected return: 0.1525635451035139
Beta portfolio expected return: 0.09099657226905389
Variance portfolio expected return: 0.060879699135729146
Res std error portfolio expected return: 0.1525635451035139
Returns portfolio expected return: -1.1418054046978832
FTSE-MIB portfolio expected return: 0.0026513448671930486


### Ratios

In [202]:
print("Beta ratio: " +
      str((final_dataframe["portfolio Weekly Returns Beta"].mean()*52)/(final_returns["R-Squared Risk"].mean()*math.sqrt(52))))
print("R-Squared ratio: " +
      str((final_dataframe["portfolio Weekly Returns R-Squared"].mean()*100*52)/(final_returns["Beta Risk"].mean()*100*math.sqrt(52))))
print("Variance ratio: " +
      str((final_dataframe["portfolio Weekly Returns Risk"].mean()*100*52)/(final_returns["Variance Risk"].mean()*100*math.sqrt(52))))
print("Error variance ratio: " +
      str((final_dataframe["portfolio Weekly Returns Res Std Error"].mean()*100*52)/(final_returns["Error Risk"].mean()*100*math.sqrt(52))))
print("Returns ratio: " +
      str((final_dataframe["portfolio Weekly Returns Returns"].mean()*100*52)/(final_returns["Returns risk"].mean()*100*math.sqrt(52))))
print("FTSE MIB ratio: " +
      str((final_dataframe["Ftse Mib weekly returns"].mean()*100*52)/(final_dataframe["FTSE_MIB Risk"].mean()*100*math.sqrt(52))))


Beta ratio: 0.2799028590727641
R-Squared ratio: 0.42106690164594
Variance ratio: 0.17266517143818003
Error variance ratio: 0.43484223756108886
Returns ratio: -4.170407776705002
FTSE MIB ratio: 0.03072194128568657


### Portfolios Rations

In [203]:
# first = True
# final_returns = pd.DataFrame(columns=["Week","Portfolio_Ret", "FTSE_MIB_Ret"])
# for index, row in final_dataframe.iterrows():
  
  
#   p_perc_return = (math.exp(row["portfolio Weekly Returns"]))
#   i_perc_return = (math.exp(row["Ftse Mib weekly returns"]))
#   if row["portfolio Weekly Returns"] < 0:
#     p_perc_return *=-1
#   if row["Ftse Mib weekly returns"]< 0:
#     i_perc_return *=-1
#   portfolio_total_return = initial_investment * \
#       p_perc_return if first else portfolio_total_return*p_perc_return
#   index_total_return = initial_investment * \
#       i_perc_return if first else index_total_return*i_perc_return
#   first = False
#   final_returns.loc[len(final_returns)] = [row["Week"],portfolio_total_return, index_total_return]

## Returns charts

In [206]:
fig = px.line(final_returns, x="Week", y=["Beta Portfolio", "R-Squared Portfolio", "Variance Portfolio", "Residual variance Portfolio", "Momentum Portfolio", "FTSE_MIB"],
              title='Weekly returns, for an investment of 100€', labels=dict(value="Returns(€)", variable="Portfolio") )
fig.show()
